# [LVV-T2261](https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2261)
**Update Me**

Created by Bruno Quint, updated by Holger Drass

This notebook is to determine the rotator torques at different rotator angles with the camera hexapod at extreme positions of its space envelope.

This notebook is considered to be used once the refrigitation lines are not pressurezied and each time a paar of lines get pressurized. 

Use the "setup" notebook to control the state machine of the components.

Make 30 degree steps to allow the rotator to arrive to full speed = full torque value 

The analysis is done in a separate notebook.

## Start Up

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
%matplotlib inline
import os
import sys
import asyncio
import logging

import numpy as np
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

import os
import pandas as pd

from astropy.time import Time
from scipy.signal import argrelextrema

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.mtcs import MTCS

from datetime import datetime, timedelta

In [ ]:
summit = 1 #use this for summit testing
# summit = 0 #use this for NCSA

## Check environment setup

The following cell will print some of the basic DDS configutions.

In [ ]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])
print(os.environ.get("LSST_DDS_DOMAIN_ID", "Expected, not set."))

### Setup logging

Setup logging in debug mode and create a logger to use on the notebook.

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

### Starting communication resources

We start by creating a domain and later instantiate the MTCS class.
We will use the class to startup the components. 

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
script = salobj.Controller("Script", index=42658886)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too

In [ ]:
await script.start_task

# Test start

## Iteration 1: Move the hexapod to (X=0, Y=0, Z=0, U=0, V=0, W=0)

In [ ]:
now = datetime.now()
test_message = "LVV-2261: Hexapod at 0 - Pathfinder Lines Not Pressurized"
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [0, 30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 2: Move the hexapod to (X=0, Y=0, Z=0, U=-0.36 deg, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod u=-0.36deg - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
#Moving the hexapod in smaller steps here at the beginning of the test to make sure no cooling line gets damaged.
for _u in [-0.08, -0.16, -0.32, -0.36]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 3: u=-0.253 v=-0.253

In [ ]:
test_message = "LVV-2261: Hexapod u=-0.253deg v=-0.253deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.253, v=-0.253, w=0, sync=True)
await asyncio.sleep(15.)  # Just so if we need to press the e-stops
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 4: u=0.36 v=0.00

In [ ]:
test_message = "LVV-2261: Hexapod u=+0.36deg v=0.00deg - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

for _v in [-0.25, -0.14, 0.00]:
    print(f" {datetime.now()} - Moving Hexapod to v={_v}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.25, v=_v, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")
    
for _u in [-0.25, -0.14, 0.00, 0.08, 0.16, 0.32, 0.36]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 5: u=-0.253deg v=+0.253deg

In [ ]:
test_message = "LVV-2261: Hexapod u=-0.253deg v=+0.253deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

for _u in [0.36, 0.253, 0.12, 0, -0.12, -0.253]:
    print(f" {datetime.now()} - Moving Hexapod to u={_u}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=_u, v=0, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")
    
for _v in [0.12, 0.253]:
    print(f" {datetime.now()} - Moving Hexapod to v={_v}")
    await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=-0.253, v=_v, w=0, sync=True)
    await asyncio.sleep(15.)  # Just so if we need to press the e-stops
    print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 6: Move the hexapod to (X=0, Y=0, Z=0, U=+0.253deg, V=+0.253deg, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=+0.253deg, V=+0.253deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=+0.253deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=+0.253, v=+0.253, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 7: Move the hexapod to (X=0, Y=0, Z=0, U=+0.253deg, V=-0.253deg, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=+0.253deg, V=-0.253deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=+0.253deg, V=-0.253deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=+0.253, v=-0.253, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 8: Move the hexapod to (X=0, Y=0, Z=0, U=0deg, V=0.36deg, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod x=0, Y=0, Z=0,U=0, V=+0.36deg, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  x=0, Y=0, Z=0, U=0, V=0.36deg, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0.36, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 9: u=0.0deg v=-0.36deg 

In [ ]:
test_message = "LVV-2261: Hexapod u=+0deg v=-0.36deg (total 0.36) - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to u=+0deg v=-0.36deg")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=0, z=0, u=0, v=0.36, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 10: x=-11400, Y=0, Z=-10000

In [ ]:
test_message = "LVV-2261: Hexapod x=-11400, Y=0, Z=-10000 - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")

print(f" {datetime.now()} - Moving Hexapod to x=-11400, Y=0, Z=-10000")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=-11400, y=0, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 11 = Move the hexapod to (X=11400, Y=0, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=11400, Y=0, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=11400, Y=0, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=11400, y=0, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 12: Move the hexapod to (X=6700, Y=6700, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=6700, Y=6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=6700, Y=6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=6700, y=6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 13: Move the hexapod to (X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0

In [ ]:
test_message = "LVV-2261: Hexapod X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=6700, Y=-6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=6700, y=-6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 14: Move the hexapod to (X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=-6700, Y=-6700, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=-6700, y=-6700, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 16: Move the hexapod to (X=0, Y=-11400, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=0, Y=-10000, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=0, Y=-10000, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=-10000, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")

## Iteration 17: Move the hexapod to (X=0, Y=11400, Z=-10000, U=0, V=0, W=0)

In [ ]:
test_message = "LVV-2261: Hexapod X=0, Y=11400, Z=-10000, U=0, V=0, W=0  - Pathfinder Lines Not Pressurized"

now = datetime.now()
script.log.info(f"START -- {test_message} -- Starting Time: {now} UTC")
print(f" {datetime.now()} - Moving Hexapod to  X=0, Y=11400, Z=-10000, U=0, V=0, W=0")
await mtcs.rem.mthexapod_1.cmd_move.set_start(x=0, y=11400, z=-10000, u=0, v=0, w=0, sync=True)
print(f" Done.")

In [ ]:
for p in [30, 60, 90, 60, 30, 0, -30, -60, -90, -60, 0]:
    print(f" {datetime.now()} - Moving to {p}")
    await mtcs.rem.mtrotator.cmd_move.set_start(position=p, timeout=5.)
    await asyncio.sleep(20.)
    print(f" Done.")

In [ ]:
now = datetime.now()
script.log.info(f"END -- {test_message} -- End Time: {now} UTC")